# Load Dataset

In [35]:
import pandas as pd 

df = pd.read_csv("WELFAKE_Dataset.csv")
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [36]:
df.shape

(72134, 4)

In [37]:
df = df.sample(n=5000,random_state=42).reset_index(drop=True)

In [38]:
df['label'].value_counts()

label
1    2528
0    2472
Name: count, dtype: int64

# Text Cleaning

In [39]:
import re

def clean_text(text):
    # Convert to string first
    text = str(text).lower()  
    
    # Remove unwanted characters
    text = re.sub(r"[^a-z0-9\s]", '', text)   
    
    # Remove multiple spaces
    text = " ".join(text.split())
    
    return text

df['clean_text'] = df['text'].apply(clean_text)


# Train Test Split

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test = train_test_split(df['clean_text'],df['label'], test_size=0.2, random_state=42)

# Text Transformation (Feature Extraction)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


# Train And Evaluate Models

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

models = {
    'Logistic_Regression': LogisticRegression(),
    'Random_Forest': RandomForestClassifier(),
    'SVC': SVC()
}

for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)

    print(f"-----------{name}-----------\n")
    print("Classification Report \n", classification_report(y_test, y_pred))
    print("Confusion Matrix \n", confusion_matrix(y_test, y_pred))
    print("\n\n")

-----------Logistic_Regression-----------

Classification Report 
               precision    recall  f1-score   support

           0       0.90      0.91      0.90       471
           1       0.92      0.91      0.91       529

    accuracy                           0.91      1000
   macro avg       0.91      0.91      0.91      1000
weighted avg       0.91      0.91      0.91      1000

Confusion Matrix 
 [[429  42]
 [ 49 480]]



-----------Random_Forest-----------

Classification Report 
               precision    recall  f1-score   support

           0       0.84      0.93      0.88       471
           1       0.93      0.84      0.89       529

    accuracy                           0.89      1000
   macro avg       0.89      0.89      0.88      1000
weighted avg       0.89      0.89      0.89      1000

Confusion Matrix 
 [[438  33]
 [ 82 447]]



-----------SVC-----------

Classification Report 
               precision    recall  f1-score   support

           0       0.9

# Selecting Best Model

In [43]:
model = SVC()

model.fit(X_train_tfidf,y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [49]:
# Prediction 
text = 'And now a message of peace and unity from one of our neighbors to the South:  We, Mexicans, have to kill Donald J. Trump before he becomes President. He is a threat to every single one of us. There are many Mexican Americans living in the U.S. right now and I m asking them to kill Donald Trump before he becomes President. The one in Mexico who have the means, I m asking you to cross the border and go and kill Donald Trump, and as many of his supporters as possible. Anywhere he goes just try to bomb the place, shoot up the place, do something. B..b but he looked like such a nice boy. Don t call him  illegal he just wanted a better life he s just a victim This punk isn t the only one threatening the life of Trump. Watch this video that exposes the truth about how the media ignores these threats:'
text = clean_text(text)
text = tfidf.transform([text])
print(model.predict(text)[0])

1


In [51]:
# Prediction 
text = 'WINNIPEG, Manitoba (Reuters) - Former U.S. President Jimmy Carter, appearing fully recovered from dehydration suffered while helping to build a home for charity in Canada, was released from an overnight hospital stay on Friday and addressed the project’s closing ceremony. Carter, 92, collapsed while working on Thursday at the Winnipeg construction site for Habitat for Humanity, which promotes affordable home ownership, and was taken to St. Boniface General Hospital for medical treatment and tests. By Friday morning, Carter was smiling as he returned to the building site to help kick off the project’s last day. Hours later, he and his wife, Rosalynn Carter, 89, attended closing ceremonies at the Canadian Museum for Human Rights in Manitoba’s capital, receiving a rousing ovation from the crowd. Dressed in blue jeans, a T-shirt and light-weight jacket, a relaxed, fit-looking Carter climbed a short flight of steps to the stage to salute Habitat’s members for their contributions. “I look upon all the volunteers, in a very sincere way, as human rights heroes, and I thank you for it,” he said, and joked that his “bringing attention to this Habitat project was completely unintentional.” The former first lady told the crowd her husband received a clean bill of health after an extensive battery of tests during his brief hospitalization, including from one test designed to detect heart damage. The results showed “there has never been any kind of damage at all to Jimmy Carter’s heart,” she said. “I knew he had a good heart.” Thursday’s health scare generated an outpouring of support for Carter, a Democrat who served in the White House from January 1977 to January 1981 and has lived longer after his term in office than any other president in U.S. history. A high point of his presidency was Carter’s role in brokering the 1978 Camp David Accords that ushered in peace between Israel and Egypt.  But having left office profoundly unpopular, he is widely regarded as a better former president than he was a president, and received the Nobel Peace Prize in 2002 for his humanitarian work. Carter disclosed in August 2015 that he had been diagnosed with a form of skin cancer called melanoma that had spread to his brain and elsewhere and had been spotted during liver surgery. But months later, Carter told the Maranatha Baptist Church, where he teaches Sunday school in his home town of Plains, Georgia, that his latest brain scan showed no sign of the disease. Carter was in Canada for a project to build 150 new homes for needy families, celebrating the country’s 150th independence anniversary. '
text = clean_text(text)
text = tfidf.transform([text])
print(model.predict(text)[0])

0


In [46]:
# Save
import pickle
pickle.dump(tfidf, open("tfidf.pkl",'wb'))
pickle.dump(model, open("model.pkl",'wb'))

In [50]:
df['text'][2]

'WINNIPEG, Manitoba (Reuters) - Former U.S. President Jimmy Carter, appearing fully recovered from dehydration suffered while helping to build a home for charity in Canada, was released from an overnight hospital stay on Friday and addressed the project’s closing ceremony. Carter, 92, collapsed while working on Thursday at the Winnipeg construction site for Habitat for Humanity, which promotes affordable home ownership, and was taken to St. Boniface General Hospital for medical treatment and tests. By Friday morning, Carter was smiling as he returned to the building site to help kick off the project’s last day. Hours later, he and his wife, Rosalynn Carter, 89, attended closing ceremonies at the Canadian Museum for Human Rights in Manitoba’s capital, receiving a rousing ovation from the crowd. Dressed in blue jeans, a T-shirt and light-weight jacket, a relaxed, fit-looking Carter climbed a short flight of steps to the stage to salute Habitat’s members for their contributions. “I look u